In [2]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
from dotenv import load_dotenv

app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

embedding_function = OpenAIEmbeddings()

docs = [
    Document(
        page_content="the dog loves to eat pizza", metadata={"source": "animal.txt"}
    ),
    Document(
        page_content="the cat loves to eat lasagna", metadata={"source": "animal.txt"}
    ),
]


db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

In [3]:
retriever.invoke("What exactly?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}),
 Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'})]

In [4]:
from langchain.prompts.prompt import PromptTemplate

rephrase_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
REPHRASE_TEMPLATE = PromptTemplate.from_template(rephrase_template)

In [5]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

rephrase_chain = REPHRASE_TEMPLATE | ChatOpenAI(temperature=0) | StrOutputParser()

In [6]:
rephrase_chain.invoke(
    {
        "question": "No, really?",
        "chat_history": [
            HumanMessage(content="What does the dog like to eat?"),
            AIMessage(content="Thuna!"),
        ],
    }
)

'Is it really true that the dog likes to eat tuna?'

In [7]:
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [8]:
from langchain_core.runnables import RunnablePassthrough

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | ANSWER_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

In [9]:
final_chain = rephrase_chain | retrieval_chain

In [10]:
final_chain.invoke(
    {
        "question": "No, really?",
        "chat_history": [
            HumanMessage(content="What does the dog like to eat?"),
            AIMessage(content="Thuna!"),
        ],
    }
)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'Based on the provided context, it is not mentioned that the dog likes to eat tuna. The dog loves to eat pizza.'

### Chat with returning documents

In [11]:
retrieved_documents = {"docs": retriever, "question": RunnablePassthrough()}
final_inputs = {
    "context": lambda x: "\n".join(doc.page_content for doc in x["docs"]),
    "question": RunnablePassthrough(),
}
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI() | StrOutputParser(),
    "docs": RunnablePassthrough(),
}

final_chain = rephrase_chain | retrieved_documents | answer

In [12]:
result = final_chain.invoke(
    {
        "question": "No, really?",
        "chat_history": [
            HumanMessage(content="What does the dog like to eat?"),
            AIMessage(content="Thuna!"),
        ],
    }
)
print(result)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'answer': 'No, based on the given context, it is not true that the dog likes to eat tuna. The dog loves to eat pizza.', 'docs': {'docs': [Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}), Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'})], 'question': 'Is it really true that the dog likes to eat tuna?'}}


In [13]:
result["answer"]

'No, based on the given context, it is not true that the dog likes to eat tuna. The dog loves to eat pizza.'

In [14]:
result["docs"]["docs"]

[Document(page_content='the dog loves to eat pizza', metadata={'source': 'animal.txt'}),
 Document(page_content='the cat loves to eat lasagna', metadata={'source': 'animal.txt'})]